In [ ]:
using Turing
using Random
using Statistics
using Distributions
using StatsPlots
using LaTeXStrings
using DataFrames
using FFTW
using Dierckx

In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfontsize=12, lw=2, ms=8)

In [ ]:
N = 32;
γ = 0.01;
x = LinRange(0,1,N+1)[2:end-1];

# true value that we wish to recover
# x_data = [0.2, 0.4, 0.6, 0.8];
# n_data = length(x_data);
uᵗ(x) = x*(1-x);

Random.seed!(500); # set a seed for reproducibility
y_data = @. uᵗ(x) + γ * randn()

In [ ]:
plot(x, uᵗ.(x))
scatter!(x, y_data)

In [ ]:
"""
`build_field` - Build a mean zero Gaussian random field with the (-Δ)^{-α} covariance operator in dimension one

### Fields
`ξ`   - Vector of N(0,1) values  
`α=1` - Smoothness parameter
"""
function build_field(ξ; α=1)
    N = length(ξ)
    
    uhat = zeros(ComplexF64,2*N); # preallocate space

    # construct the eigenvalues
    k = 1:N;
    λ = @. 1/(π*k)^(2*α);

    # fill in the nonzero entries
    # NOTE we need to multiply by 2 *N for FFT scaling
    @. uhat[2:N+1] = 2 * N * sqrt(λ) * sqrt(2) * ξ;

    # invert and get the relevant imaginary part
    u = imag.(ifft(uhat))[N+2:end];
    return u
end

In [ ]:
@model function mean_recovery(y_data)
    ξ ~ MvNormal(zeros(N), 1.)
    u = build_field(ξ);
    
    for i in 1:length(y_data)
       y_data[i]~Normal(u[i], γ^2)
    end
    
end

In [ ]:
model=mean_recovery(y_data)

In [ ]:
chain = sample(model, HMC(0.1, 10), 10^4)